# Cybersecurity advanced
## Lab 08: Siem



Links to Syllabus

- [Theoretical](https://hogenttin.github.io/cybersecurity-advanced/lesson-8/theory/)
- [Practical](https://hogenttin.github.io/cybersecurity-advanced/lesson-8/lab/)

Creation of VM:

```vagrant
config.vm.define "wazuh" do |host|
        host.vm.box = "almalinux/9"
        host.vm.hostname = "wazuh"

        host.vm.network "private_network", ip: "172.30.0.20", netmask: "255.255.255.0", virtualbox__intnet: "internal-company-lan"

        host.vm.provider :virtualbox do |v|
            v.customize ["modifyvm", :id, "--groups", "/CSA"]
            v.name = "wazuh"
            v.cpus = "2"
            v.memory = "6334"
        end

        host.vm.provision "shell", inline: <<-SHELL
            # Update system
            yum update -y
            # Install dependencies
            yum install -y curl unzip wget
            # Download and install Wazuh repository
            rpm --import https://packages.wazuh.com/key/GPG-KEY-WAZUH
            echo -e '[wazuh]\ngpgcheck=1\ngpgkey=https://packages.wazuh.com/key/GPG-KEYWAZUH\nenabled=1\nname=EL-$releasever -Wazuh\nbaseurl=https://packages.wazuh.com/4.x/yum/\nprotect=1' | tee /etc/yum.repos.d/wazuh.repo

            # Install Wazuh manager
            yum install -y wazuh-manager
            # Start and enable Wazuh
            systemctl enable wazuh-manager
            systemctl start wazuh-manager
            # Configure default gateway
            nmcli connection modify "System eth1" ipv4.gateway 172.30.255.254
            systemctl restart NetworkManager
        SHELL
    end
```

`vagrant ssh wazuh`
→ add cyber.pub to authorized_keys

disconnect nat interface

SSH via config: `ssh wazuh`

Installation of Wazuh

```
curl -sO https://packages.wazuh.com/4.x/wazuh-install.sh 
curl -sO https://packages.wazuh.com/4.x/config.yml
```

Change config-file:
```bash
nodes: 
  # Wazuh indexer nodes 
  indexer: 
    - name: node-1 
      ip: 172.30.0.20 
 
  # Wazuh server nodes 
  # If there is more than one Wazuh server 
  # node, each one must have a node_type 
  server: 
    - name: wazuh-1 
      ip: 172.30.0.20 
 
  # Wazuh dashboard nodes 
  dashboard: 
    - name: dashboard 
      ip: 172.30.0.20
```

Execute installation commands:
```bash
sudo bash wazuh-install.sh --generate-config-files -i 

sudo tar -xf wazuh-install-files.tar 
cd wazuh-install-files/
sudo cat wazuh-passwords.txt

sudo bash wazuh-install.sh --wazuh-indexer node-1 -i 

sudo bash wazuh-install.sh --start-cluster -i

sudo bash wazuh-install.sh --wazuh-server wazuh-1 -i -o 

sudo bash wazuh-install.sh --wazuh-dashboard dashboard -i
```

To display the dashboard on the host device: `ssh -L 443:172.30.0.20:443 cyber-wazuh`

Agents
- AlmaLinux installation
```bash
sudo rpm --import https://packages.wazuh.com/key/GPG-KEY-WAZUH 
sudo cat > /etc/yum.repos.d/wazuh.repo << EOF 
[wazuh] 
gpgcheck=1 
gpgkey=https://packages.wazuh.com/key/GPG-KEY-WAZUH 
enabled=1 
name=EL-\$releasever - Wazuh 
baseurl=https://packages.wazuh.com/4.8/yum/ 
protect=1 
EOF 
sudo WAZUH_MANAGER="172.30.0.20" yum install wazuh-agent 
 
sudo systemctl daemon-reload 
sudo systemctl enable wazuh-agent 
sudo systemctl start wazuh-agent 
sudo sed -i "s/^enabled=1/enabled=0/" /etc/yum.repos.d/wazuh.repo
```
- Alpine Installation
```bash
dns:~$ sudo su 
dns:~# wget -O /etc/apk/keys/alpine-devel@wazuh.com-633d7457.rsa.pub https://packages.wazuh.com/key/alpine-devel%40wazuh.com-633d7457.rsa.pub 
 
dns:~$ echo "https://packages.wazuh.com/4.x/alpine/v3.12/main" >> /etc/apk/repositories 
dns:~# apk update 

dns:~# apk add wazuh-agent=4.8.2-r1 

dns:~# export WAZUH_MANAGER="172.30.0.20" && sed -i 
 
dns:~# sed -i "s|^https://packages.wazuh.com|#https://packages.wazuh.com|g" /etc/apk/repositories
```

File Integrity:

Change the ossec.conf file in the /var/ossec/etc directory (change to root: `sudo su`)
add in the directories part following line: `<directories realtime="yes">/home/vagrant</directories>`
restart agent: `sudo systemctl restart wazuh-agent`

# Demo

`ssh -L 443:172.30.0.20:443 wazuh`

localhost:443

inlog wazuh dashboard:

- login admin

- password: kU.53gP2LHhJ0U94+yl3GkIK8RHq+NqX


New process = auditd, not implemented

